# **Prediccion de la produccion vs viento**

## Modelos lineales, utilizando datos historicos para entrenar y datos de comportamiento futuro del viento para calcular la produccion

### ***Modelo LinearGAM***

In [ ]:
!pip install pygam

import pandas as pd
from pygam import LinearGAM, s

# --- Cargar datos históricos ---
df = pd.read_csv("/content/tablaresumen.csv")
df = df.drop(columns=['Unnamed: 0'])

# Variable predictora y target
X = df[["v_promedio"]]
y = df["Prod"]

# Ajustar el modelo GAM usando solo v_promedio
gam = LinearGAM(s(0)).fit(X, y)

# --- Cargar datos futuros de viento diario ---
df_viento = pd.read_csv("/content/Vientos_predecidosarima.csv")
df_viento['fecha'] = pd.to_datetime(df_viento['fecha'])
df_viento['Año'] = df_viento['fecha'].dt.year

# Promedio anual de viento
viento_anual = df_viento.groupby('Año')['velocidad_viento'].mean().reset_index()
viento_anual.rename(columns={'velocidad_viento':'v_promedio'}, inplace=True)

# Filtrar años 2024-2027
viento_futuro = viento_anual[(viento_anual['Año'] >= 2024) & (viento_anual['Año'] <= 2027)]

# Preparar variable para predicción
X_futuro = viento_futuro['v_promedio'].values.reshape(-1,1)

# Predicción GAM 2024-2027
pred_futuro_gam = gam.predict(X_futuro)

# Combinar resultados en DataFrame
df_pred_gam = viento_futuro.copy()
df_pred_gam['Pred_Prod'] = pred_futuro_gam

print(df_pred_gam)

# Opcional: guardar a CSV
df_pred_gam.to_csv("/content/Prediccion_GAM_2024_2027.csv", index=False)
gam = LinearGAM(s(0)).fit(X, y)

# R² del modelo
r2_gam = gam.statistics_['pseudo_r2']['explained_deviance']

print(f"R² del modelo GAM (solo v_promedio): {r2_gam:.3f}")


     Año  v_promedio      Pred_Prod
10  2024    1.942309  167632.108668
11  2025    2.208246  156447.701072
12  2026    1.824742  152834.480589
13  2027    1.856703  155796.444149
R² del modelo GAM (solo v_promedio): -0.199


***Resultados de Quintales de maiz que se produciran***

In [ ]:
df_pred_gam


,Año,v_promedio,Pred_Prod
10,2024,1.942309,167632.108668
11,2025,2.208246,156447.701072
12,2026,1.824742,152834.480589
13,2027,1.856703,155796.444149


### ***Modelo de regresion lineal***

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# --- Cargar datos de producción ---
df_prod = pd.read_csv("/content/tablaresumen.csv")
df_prod = df_prod.drop(columns=['Unnamed: 0'])

# Variables para el modelo: velocidad promedio del viento vs producción
X = df_prod[["v_promedio"]]
y = df_prod["Prod"]

# Crear y entrenar el modelo lineal
model = LinearRegression()
model.fit(X, y)

# --- Cargar datos de viento diario ---
df_viento = pd.read_csv("/content/Vientos_predecidosarima.csv")
df_viento['fecha'] = pd.to_datetime(df_viento['fecha'])

# Calcular promedio anual
df_viento['Año'] = df_viento['fecha'].dt.year
viento_anual = df_viento.groupby('Año')['velocidad_viento'].mean().reset_index()
viento_anual.rename(columns={'velocidad_viento':'v_promedio'}, inplace=True)

# Filtrar solo años 2024 a 2027
viento_futuro = viento_anual[(viento_anual['Año'] >= 2024) & (viento_anual['Año'] <= 2027)]

# Predicción de producción 2024-2027
X_futuro = viento_futuro['v_promedio'].values.reshape(-1,1)
pred_futuro = model.predict(X_futuro)

# Combinar resultados en un DataFrame
df_pred = viento_futuro.copy()
df_pred['Pred_Prod'] = pred_futuro

print(df_pred)
df_pred.to_csv("/content/Prediccion_Lineal_2024_2027.csv", index=False)

# R² del modelo
r2 = model.score(X, y)

print(f"R² del modelo lineal (v_promedio vs Prod): {r2:.3f}")

     Año  v_promedio      Pred_Prod
10  2024    1.942309  158943.539479
11  2025    2.208246  149739.740768
12  2026    1.824742  163012.422067
13  2027    1.856703  161906.256945
R² del modelo lineal (v_promedio vs Prod): 0.045


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


***Resultados de Quintales de maiz que se produciran***

In [ ]:
df_pred

,Año,v_promedio,Pred_Prod
10,2024,1.942309,158943.539479
11,2025,2.208246,149739.740768
12,2026,1.824742,163012.422067
13,2027,1.856703,161906.256945


# **Prediccion de la produccion vs multivariables**


## Proyección lineal basada en crecimiento promedio anual. de datos historicos para predecir la produccion con multivariables.

In [ ]:
import pandas as pd

# Datos históricos
data = {
    "Año": [2016,2017,2018,2019,2020,2021,2022,2023],
    "Provincia": ["Los Santos"]*8,
    "Prod": [103280,170110,156490,151240,161860,165290,170690,203920],
    "Siembra": [19050,16710,19950,20320,20560,23020,20770,20054],
    "avg_temp_c": [27.055175,27.43913043,27.72251462,28.12439759,28.26457565,22.07678571,27.35882353,28.4],
    "LluviaDiariamm": [4.018075802,4.284386617,4.426,4.325825826,3.080882353,2.431764706,5.605154639,5.441134752],
    "v_promedio": [1.984577236,1.921138875,1.73111687,1.908536909,2.212173084,1.651437138,1.972274547,1.82979681],
    "Vinetos_nacional": [5.449423039,5.530521175,5.356496615,5.598550439,5.894996045,5.274629266,5.209159571,5.371658395]
}

df = pd.DataFrame(data)

# Calcular crecimiento promedio anual (excepto v_promedio)
cols_to_project = ["Prod", "Siembra", "avg_temp_c", "LluviaDiariamm", "Vinetos_nacional"]
crecimiento_anual = {}
for col in cols_to_project:
    crecimiento_anual[col] = (df[col].iloc[-1] - df[col].iloc[0]) / (df.shape[0]-1)

# Proyección para 4 años futuros
proyecciones = []
ult_ano = df["Año"].iloc[-1]
ult_values = df.iloc[-1]

for i in range(1,5):
    nuevo_ano = ult_ano + i
    fila = {"Año": nuevo_ano, "Provincia": "Los Santos"}
    for col in cols_to_project:
        fila[col] = ult_values[col] + crecimiento_anual[col]*i
        fila[f"Crecimiento anual {col}"] = crecimiento_anual[col]
    # Mantener v_promedio constante
    fila["v_promedio"] = ult_values["v_promedio"]
    # Calcular rendimiento
    fila["Rendimiento"] = fila["Prod"] / fila["Siembra"]

    proyecciones.append(fila)

df_proyecciones = pd.DataFrame(proyecciones)

# Reordenar columnas para mejor visualización
cols_order = ["Año","Provincia","Prod","Crecimiento anual Prod","Siembra","Crecimiento anual Siembra",
              "Rendimiento","avg_temp_c","Crecimiento anual avg_temp_c",
              "LluviaDiariamm","Crecimiento anual LluviaDiariamm","v_promedio",
              "Vinetos_nacional","Crecimiento anual Vinetos_nacional"]

df_proyecciones = df_proyecciones[cols_order]

# Mostrar resultados
print(df_proyecciones)

### **Modelo LinearGAM multivariable**

In [ ]:
import pandas as pd
from pygam import LinearGAM, s, te

# --- 1. Cargar datos ---
df_hist = pd.read_csv("/content/tablaresumen.csv").drop(columns=['Unnamed: 0'], errors='ignore')
df_proyecciones = pd.read_csv("/content/proyecciones.csv")
df_proyecciones=df_proyecciones.drop(columns=['v_promedio'])
df_viento = pd.read_csv("/content/Vientos_predecidosarima.csv")

# --- 2. Procesar v_promedio a partir de vientos ---
df_viento['fecha'] = pd.to_datetime(df_viento['fecha'])
df_viento['Año'] = df_viento['fecha'].dt.year
viento_anual = df_viento.groupby('Año')['velocidad_viento'].mean().reset_index()
viento_anual.rename(columns={'velocidad_viento':'v_promedio'}, inplace=True)

# --- 3. Agregar v_promedio a df_proyecciones ---
df_proyecciones = df_proyecciones.merge(viento_anual, how='left', on='Año')

# --- 4. Definir columnas del modelo ---
cols_modelo = ["Siembra", "avg_temp_c", "LluviaDiariamm", "v_promedio"]

# --- 5. Preparar variables históricas ---
X_hist = df_hist[cols_modelo].values
y_hist = df_hist["Prod"].values

# --- 6. Ajustar GAM flexible ---
gam = LinearGAM(
    s(0, n_splines=30) + s(1, n_splines=30) + s(2, n_splines=30) + s(3, n_splines=30)  +
    te(0,1) + te(1,2) + te(2,3)
).gridsearch(X_hist, y_hist)  # optimiza suavidad automáticamente

# --- 7. Preparar datos de proyección ---
X_proj = df_proyecciones[cols_modelo].values

# --- 8. Predecir producción ---
y_pred = gam.predict(X_proj)

# --- 9. Obtener intervalos de confianza ---
intervalos = gam.prediction_intervals(X_proj, width=0.95)
df_proyecciones["Prod_predicha"] = y_pred
df_proyecciones["Prod_lower"] = intervalos[:,0]
df_proyecciones["Prod_upper"] = intervalos[:,1]

# --- 10. Guardar resultados ---
df_proyecciones.to_csv("/content/proyecciones_con_predicciones.csv", index=False)

print("Predicciones completadas con GAM flexible e intervalos de confianza.")


100% (11 of 11) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


Predicciones completadas con GAM flexible e intervalos de confianza.


Resultados

In [ ]:
resultados = df_proyecciones
resultados= resultados[['Año','Prod_predicha']]
resultados

,Año,Prod_predicha
0,2024,209913.874704
1,2025,198611.869420
2,2026,269674.984013
3,2027,291239.993500


### **Modelo red neuronal con Keras**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# --- 1. Cargar datos ---
df_hist = pd.read_csv("/content/tablaresumen.csv").drop(columns=['Unnamed: 0'], errors='ignore')
df_proyecciones = pd.read_csv("/content/proyecciones.csv")
df_proyecciones = df_proyecciones.drop(columns=['v_promedio'], errors='ignore')
df_viento = pd.read_csv("/content/Vientos_predecidosarima.csv")

# --- 2. Procesar v_promedio a partir de vientos ---
df_viento['fecha'] = pd.to_datetime(df_viento['fecha'])
df_viento['Año'] = df_viento['fecha'].dt.year
viento_anual = df_viento.groupby('Año')['velocidad_viento'].mean().reset_index()
viento_anual.rename(columns={'velocidad_viento':'v_promedio'}, inplace=True)

# --- 3. Agregar v_promedio a df_proyecciones ---
df_proyecciones = df_proyecciones.merge(viento_anual, how='left', on='Año')

# --- 4. Definir columnas del modelo ---
cols_modelo = ["Siembra", "avg_temp_c", "LluviaDiariamm", "v_promedio"]

# --- 5. Preparar variables históricas ---
X_hist = df_hist[cols_modelo].values
y_hist = df_hist["Prod"].values

# --- 6. Escalar los datos ---
scaler = StandardScaler()
X_hist_scaled = scaler.fit_transform(X_hist)
X_proj_scaled = scaler.transform(df_proyecciones[cols_modelo].values)

# --- 7. Crear la red neuronal ---
model = Sequential([
    Dense(64, input_dim=X_hist_scaled.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # salida de producción
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# --- 8. Entrenar la red ---
model.fit(X_hist_scaled, y_hist, epochs=200, batch_size=8, verbose=0)

# --- 9. Predecir producción ---
y_pred = model.predict(X_proj_scaled).flatten()
df_proyecciones["Prod_predicha"] = y_pred

# --- 10. Guardar resultados ---
df_proyecciones.to_csv("/content/proyecciones_nn_prediccionesnn.csv", index=False)

print("Predicciones completadas con red neuronal.")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Predicciones completadas con red neuronal.


In [ ]:
# Predicciones sobre datos históricos
y_train_pred = model.predict(X_hist_scaled).flatten()

# Métricas
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

r2 = r2_score(y_hist, y_train_pred)
mse = mean_squared_error(y_hist, y_train_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_hist, y_train_pred)

print("R²:", r2)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
R²: -37.48173141479492
MSE: 26369980416.0
RMSE: 162388.3629328161
MAE: 160266.1875


Resultados

In [ ]:
resultados = pd.read_csv("/content/proyecciones_nn_prediccionesnn.csv")
resultados= resultados[['Año','Prod_predicha']]
resultados

,Año,Prod_predicha
0,2024,84.11299
1,2025,104.77387
2,2026,107.44951
3,2027,114.69582


### **Modelo Polinomial**

In [ ]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# --- 1. Cargar datos ---
df_hist = pd.read_csv("/content/tablaresumen.csv").drop(columns=['Unnamed: 0'], errors='ignore')
df_proyecciones = pd.read_csv("/content/proyecciones.csv")
df_proyecciones = df_proyecciones.drop(columns=['v_promedio'], errors='ignore')
df_viento = pd.read_csv("/content/Vientos_predecidosarima.csv")

# --- 2. Procesar v_promedio a partir de vientos ---
df_viento['fecha'] = pd.to_datetime(df_viento['fecha'])
df_viento['Año'] = df_viento['fecha'].dt.year
viento_anual = df_viento.groupby('Año')['velocidad_viento'].mean().reset_index()
viento_anual.rename(columns={'velocidad_viento':'v_promedio'}, inplace=True)

# --- 3. Agregar v_promedio a df_proyecciones ---
df_proyecciones = df_proyecciones.merge(viento_anual, how='left', on='Año')

# --- 4. Definir columnas del modelo ---
cols_modelo = ["Siembra", "avg_temp_c", "LluviaDiariamm", "v_promedio"]

# --- 5. Preparar variables ---
X_hist = df_hist[cols_modelo].values
y_hist = df_hist["Prod"].values
X_proj = df_proyecciones[cols_modelo].values

# --- 6. Transformación polinómica ---
poly = PolynomialFeatures(degree=2, include_bias=False)  # grado 2 para interacciones y cuadrados
X_hist_poly = poly.fit_transform(X_hist)
X_proj_poly = poly.transform(X_proj)

# --- 7. Ajustar regresión lineal sobre polinomios ---
model = LinearRegression()
model.fit(X_hist_poly, y_hist)

# --- 8. Predecir producción ---
y_pred = model.predict(X_proj_poly)
df_proyecciones["Prod_predicha"] = y_pred

# --- 9. Evaluar métricas en entrenamiento ---
y_train_pred = model.predict(X_hist_poly)
r2 = r2_score(y_hist, y_train_pred)
mse = mean_squared_error(y_hist, y_train_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_hist, y_train_pred)

print("R² entrenamiento:", r2)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)

# --- 10. Guardar resultados ---
df_proyecciones.to_csv("/content/proyecciones_polynomial.csv", index=False)
print("Predicciones completadas con regresión polinómica.")


R² entrenamiento: 1.0
MSE: 7.595936060049782e-12
RMSE: 2.756072578879189e-06
MAE: 2.347456756979227e-06
Predicciones completadas con regresión polinómica.


Resultados

In [ ]:
resultados = pd.read_csv("/content/proyecciones_polynomial.csv")
resultados= resultados[['Año','Prod_predicha']]
resultados

,Año,Prod_predicha
0,2024,209913.874704
1,2025,198611.869420
2,2026,269674.984013
3,2027,291239.993500


## **Promedio de resultados**

In [ ]:
import pandas as pd

# --- 1. Cargar los tres CSV ---
df_gam = pd.read_csv("/content/proyecciones_con_predicciones.csv")       # GAM
#df_nn = pd.read_csv("/content/proyecciones_nn_prediccionesnn.csv")          # Red neuronal
df_poly = pd.read_csv("/content/proyecciones_polynomial.csv")             # Polinomio

# --- 2. Verificar que tengan la misma longitud y orden ---
assert len(df_gam) == len(df_nn) == len(df_poly), "Los CSV no tienen la misma cantidad de filas."

# --- 3. Sacar promedio de las predicciones ---
df_promedio = pd.DataFrame()
df_promedio["Año"] = df_gam["Año"]  # o la columna que identifique cada fila
df_promedio["Prod_promedio"] = (
    df_gam["Prod_predicha"] +
    #df_nn["Prod_predicha"] +
    df_poly["Prod_predicha"]
) / 2

# --- 4. Guardar resultado ---
df_promedio.to_csv("/content/proyecciones_promedio.csv", index=False)

print("Promedio de las tres predicciones guardado en 'proyecciones_promedio.csv'")


Promedio de las tres predicciones guardado en 'proyecciones_promedio.csv'


In [ ]:
resultados = pd.read_csv("/content/proyecciones_promedio.csv")

resultados

,Año,Prod_promedio
0,2024,192838.724291
1,2025,172957.542280
2,2026,205319.446586
3,2027,202633.580607


Produccion con reduccion del 25% si el viento supera el umbral de los 7m/s


In [ ]:
import pandas as pd

# --- 1. Cargar los CSV ---
df_gam = pd.read_csv("/content/proyecciones_con_predicciones.csv")
# df_nn = pd.read_csv("/content/proyecciones_nn_prediccionesnn.csv")
df_poly = pd.read_csv("/content/proyecciones_polynomial.csv")

# --- 2. Sacar promedio de las predicciones ---
df_promedio = pd.DataFrame()
df_promedio["Año"] = df_gam["Año"]
df_promedio["Prod_promedio"] = (df_gam["Prod_predicha"] + df_poly["Prod_predicha"]) / 2

# --- 3. Simulación: reducción del 25% por viento alto ---
df_promedio["Prod_simulada_25pct"] = df_promedio["Prod_promedio"] * 0.75

# --- 4. Guardar resultado ---
df_promedio.to_csv("/content/proyecciones_promedio_simulacion.csv", index=False)



resultados = pd.read_csv("/content/proyecciones_promedio_simulacion.csv")

resultados

,Año,Prod_promedio,Prod_simulada_25pct
0,2024,192838.724291,144629.043218
1,2025,172957.542280,129718.156710
2,2026,205319.446586,153989.584939
3,2027,202633.580607,151975.185456
